# Model Building for Disease Prediction

This notebook focuses on building and comparing different models for predicting disease outcomes.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
import time
import sys
import os

# Add the src directory to path
sys.path.append(os.path.abspath('../src'))
from modeling import train_test_validation_split, evaluate_classification_model, plot_confusion_matrix, plot_roc_curve

# Visualization settings
plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)

## Load the Processed Data

In [ ]:
# TODO: Update the path to your actual processed data file
# data_path = '../data/processed_features.csv'
# df = pd.read_csv(data_path)

# For now, create a synthetic dataset
n_samples = 500
n_features = 15

# Generate synthetic data with some predictive signal
np.random.seed(42)
X = np.random.randn(n_samples, n_features)
# Make the first few features more predictive
true_coef = np.zeros(n_features)
true_coef[:5] = np.array([2.0, 1.0, 0.8, -1.5, 1.2])
y_prob = 1 / (1 + np.exp(-np.dot(X, true_coef) + np.random.randn(n_samples) * 0.5))
y = (y_prob > 0.5).astype(int)

# Create a dataframe
feature_names = [f'feature_{i}' for i in range(n_features)]
df = pd.DataFrame(X, columns=feature_names)
df['disease_status'] = y

# Show the data
print(f"Dataset shape: {df.shape}")
print(f"Positive cases: {sum(y)} ({sum(y)/len(y):.1%})")
df.head()

## Split the Data

In [ ]:
# Separate features and target
X = df.drop(columns=['disease_status'])
y = df['disease_status']

# Split into train, validation, and test sets
X_train, X_val, X_test, y_train, y_val, y_test = train_test_validation_split(
    X, y, test_size=0.2, validation_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train a logistic regression model
print("Training Logistic Regression model...")
start_time = time.time()
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)
train_time = time.time() - start_time
print(f"Training completed in {train_time:.2f} seconds")

# Evaluate on validation set
y_val_pred = lr_model.predict(X_val)
y_val_prob = lr_model.predict_proba(X_val)[:, 1]

# Print basic metrics
print(f"Validation accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Validation ROC AUC: {roc_auc_score(y_val, y_val_prob):.4f}")
print(f"Validation F1 score: {f1_score(y_val, y_val_pred):.4f}")

# Plot ROC curve
plot_roc_curve(y_val, y_val_prob, label='Logistic Regression')

# Show feature coefficients
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr_model.coef_[0]
}).sort_values('Coefficient', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(x='Coefficient', y='Feature', data=coef_df)
plt.title('Logistic Regression Coefficients')
plt.tight_layout()
plt.show()

## Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train a random forest model
print("Training Random Forest model...")
start_time = time.time()
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
train_time = time.time() - start_time
print(f"Training completed in {train_time:.2f} seconds")

# Evaluate on validation set
y_val_pred_rf = rf_model.predict(X_val)
y_val_prob_rf = rf_model.predict_proba(X_val)[:, 1]

# Print basic metrics
print(f"Validation accuracy: {accuracy_score(y_val, y_val_pred_rf):.4f}")
print(f"Validation ROC AUC: {roc_auc_score(y_val, y_val_prob_rf):.4f}")
print(f"Validation F1 score: {f1_score(y_val, y_val_pred_rf):.4f}")

# Plot confusion matrix
plot_confusion_matrix(confusion_matrix(y_val, y_val_pred_rf), class_names=['Negative', 'Positive'])

# Show feature importance
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(10))
plt.title('Random Forest Feature Importance')
plt.tight_layout()
plt.show()

## Gradient Boosting Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Train a gradient boosting model
print("Training Gradient Boosting model...")
start_time = time.time()
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
train_time = time.time() - start_time
print(f"Training completed in {train_time:.2f} seconds")

# Evaluate on validation set
y_val_pred_gb = gb_model.predict(X_val)
y_val_prob_gb = gb_model.predict_proba(X_val)[:, 1]

# Print basic metrics
print(f"Validation accuracy: {accuracy_score(y_val, y_val_pred_gb):.4f}")
print(f"Validation ROC AUC: {roc_auc_score(y_val, y_val_prob_gb):.4f}")
print(f"Validation F1 score: {f1_score(y_val, y_val_pred_gb):.4f}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred_gb))

## Compare Model Performance

In [ ]:
# Compare ROC curves
plt.figure(figsize=(10, 8))

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_val, y_val_prob)
roc_auc_lr = roc_auc_score(y_val, y_val_prob)
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {roc_auc_lr:.3f})')

# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_val, y_val_prob_rf)
roc_auc_rf = roc_auc_score(y_val, y_val_prob_rf)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_rf:.3f})')

# Gradient Boosting
fpr_gb, tpr_gb, _ = roc_curve(y_val, y_val_prob_gb)
roc_auc_gb = roc_auc_score(y_val, y_val_prob_gb)
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {roc_auc_gb:.3f})')

# Random classifier
plt.plot([0, 1], [0, 1], 'k--')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Create a summary table
models = ['Logistic Regression', 'Random Forest', 'Gradient Boosting']
y_val_probs = [y_val_prob, y_val_prob_rf, y_val_prob_gb]
y_val_preds = [y_val_pred, y_val_pred_rf, y_val_pred_gb]

results = []
for model, y_prob, y_pred in zip(models, y_val_probs, y_val_preds):
    results.append({
        'Model': model,
        'Accuracy': accuracy_score(y_val, y_pred),
        'ROC AUC': roc_auc_score(y_val, y_prob),
        'Precision': precision_score(y_val, y_pred),
        'Recall': recall_score(y_val, y_pred),
        'F1 Score': f1_score(y_val, y_pred)
    })

results_df = pd.DataFrame(results).set_index('Model')
results_df

## Hyperparameter Tuning for Best Model

Based on the above comparison, let's tune the hyperparameters for the best performing model.

In [ ]:
# Assuming Gradient Boosting is the best model (you should replace with your best model)
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

# For demonstration, use a smaller grid
param_grid_small = {
    'n_estimators': [50, 100],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 5]
}

print("Running grid search (this may take a while)...")
grid_search = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid_small,  # Using smaller grid for demonstration
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# Create the best model with tuned parameters
best_model = grid_search.best_estimator_

# Evaluate on validation set
best_val_pred = best_model.predict(X_val)
best_val_prob = best_model.predict_proba(X_val)[:, 1]

print(f"\nTuned model validation AUC: {roc_auc_score(y_val, best_val_prob):.4f}")
print(f"Tuned model validation F1: {f1_score(y_val, best_val_pred):.4f}")

## Final Evaluation on Test Set

Now that we've selected and tuned our best model, let's evaluate it on the test set.

In [ ]:
# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)
y_test_prob = best_model.predict_proba(X_test)[:, 1]

# Full evaluation
test_metrics = evaluate_classification_model(best_model, X_test, y_test)

# Print metrics
print(f"Test accuracy: {test_metrics['accuracy']:.4f}")
print(f"Test ROC AUC: {test_metrics['roc_auc']:.4f}")
print(f"Test PR AUC: {test_metrics['pr_auc']:.4f}")

print("\nTest Classification Report:")
print(test_metrics['classification_report'])

# Plot confusion matrix
plot_confusion_matrix(test_metrics['confusion_matrix'], class_names=['Negative', 'Positive'])

# Plot final ROC curve
plot_roc_curve(y_test, y_test_prob, label='Final Tuned Model')

## Save the Model

In [ ]:
import joblib

# Save the model to disk
model_path = '../models/disease_prediction_model.pkl'
joblib.dump(best_model, model_path)
print(f"Model saved to {model_path}")

# Save feature names for later use
feature_names_path = '../models/feature_names.pkl'
joblib.dump(X.columns.tolist(), feature_names_path)
print(f"Feature names saved to {feature_names_path}")

## Model Interpretation

Let's try to interpret what features are most important for our model's predictions.

In [ ]:
# Check if the best model has feature_importances_ attribute
if hasattr(best_model, 'feature_importances_'):
    # For tree-based models like Random Forest or Gradient Boosting
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(10))
    plt.title('Top 10 Feature Importances')
    plt.tight_layout()
    plt.show()
    
    # Print top features
    print("Top 10 most important features:")
    print(feature_importance.head(10))
elif hasattr(best_model, 'coef_'):
    # For linear models like Logistic Regression
    coef_df = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': best_model.coef_[0]
    }).sort_values('Coefficient', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Coefficient', y='Feature', data=coef_df.head(10))
    plt.title('Top 10 Feature Coefficients')
    plt.tight_layout()
    plt.show()
    
    # Print top features
    print("Top 10 features by coefficient magnitude:")
    print(coef_df.head(10))
else:
    print("Feature importance not directly available for this model.")

## Conclusion and Next Steps

In this notebook, we have:
- Built and compared several classification models for disease prediction
- Tuned the hyperparameters of the best-performing model
- Evaluated the final model on the test set
- Saved the model for future use
- Interpreted the most important features for prediction

Next steps:
1. Apply the model to real patient data
2. Develop a simple API or interface for model predictions
3. Monitor model performance over time
4. Consider exploring more advanced models or ensemble methods